# 000.0D -  PyUL_Floater_E Companion Notebook

PyUltraLight with N-Body Addon

Key Features

- N-Body Simulation Using Identical Syntax as Soliton IV
- Rudimentary interaction Models (Method = 1 and Method = 2)
- (X) FieldGradient now optimised with numba
- Field appropriately scaled for dynamics.
- Pre-Computed 2-Body Problem Example Scenarios (Parabola and Circular)
- Improved Movie Plotter Performance (Output to File or Inline)
- Removed Central Mass Settings, and rewrote Energy Calculation Code




In [ ]:
import os

save_path = 'FW_KL'  # Set output directory

print(save_path,": Do You Wish to Delete All Files Currently Stored In This Folder? [Y] \n")

def get_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size



cleardir = str(input())

if cleardir == 'Y':
    import shutil 
    shutil.rmtree(save_path)
    print("Folder Cleaned! \n")

    
try:
    os.mkdir(save_path)
    print(save_path,": Save Folder Created.")
except FileExistsError:
    if cleardir != 'Y':
        print("Folder Retained")
    else:
        print(save_path,": File Already Exists!")
        
print("The current size of the folder is", round(get_size(save_path)/1024**2,3), 'Mib')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation

import PyUltraLight_Floater_Advanced
import pyfftw
import os
import sys
import multiprocessing
import numpy
import numba
import h5py
from matplotlib import cm

import math

from IPython.core.display import clear_output, display
%matplotlib inline

# PyUL Under the Hood FW

In [ ]:
NS = 8 # (Min) Number of RK4 time steps for every Soliton update. 

# NS = 0 Switches on Explicit Euler Mode
# NS = 4n Switches on Explicit RK4 for n times
# For All Other NS Settings, scipy.integrate.solve_ivp with RK23 will be called.

Method = 1 # 1 = Fourier Series Interpolation, 2 = Scipy.Interpolate.RegularGridInterpolator

GridInfinityFix = False


## Parabola Injector

In [ ]:
## Init
Parabola = False

if Parabola:

    m = 200 #1 code unit is ~2.3e6 M_sol (1e-22/m_a)^1.5
    mS = 10 #Not doing much
    r = 0.5 #1 code unit is ~38 kpc x (1e-22/m_a)^0.5

    x0 = 2
    y0 = 2

    # Focal Point Is Origin
    # y^2 = 4cx + 4c^2

    c0 = 1/2*(-x0+np.sqrt(x0**2+y0**2))

    v0 = np.sqrt(m/(2*np.sqrt(x0**2+y0**2))) #Correct

    xDot0 = 1
    yDot0 = 2*c0*xDot0/(y0)

    vNorm = np.linalg.norm([xDot0,yDot0])

    xDot0 = xDot0/vNorm*v0
    yDot0 = yDot0/vNorm*v0

    BH1 = [m,[x0,y0,0],[-xDot0,-yDot0,0]]
    BH2 = [m,[-x0,-y0,0],[xDot0,yDot0,0]]
    
    particles = [BH1,BH2]


    #Soliton parameters are mass, position, velocity and phase (radians)
    soliton1 = [mS, [x0,y0,0],[-xDot0,-yDot0,0], 0]
    soliton2 = [mS, [-x0,-y0,0],[xDot0,yDot0,0], 0]

    solitonT = [mS, [0,0,0], [0,0,0], 0]

    solitonC = [15, [0,0,0],[0,0,0], 0]

    solitons = [soliton1,soliton2]
    # solitons = []

else:
    print("Parameter loading skipped in this block.")

## Circular Orbit Injector

In [ ]:
## Init
Circular = False

Ratio = 0.95

Replace = False

if Circular:

    m1 = 125 #1 code unit is ~2.3e6 M_sol (1e-22/m_a)^1.5
    
    m2 = 125
    
    x1 = 0.2095
    
    x2 = x1/m2*m1 # Ensures Com Position
    
    xC = x1+x2
    
    yDot1 = np.sqrt(m2*x1/xC**2)
    yDot2 = np.sqrt(m1*x2/xC**2)

    if Replace:
        
        BH1 = [Ratio*m1,[x1,0,0],[0,yDot1,0]]
        BH2 = [Ratio*m2,[-x2,0,0],[0,-yDot2,0]]
    
    else:
        BH1 = [m1,[x1,0,0],[0,yDot1,0]]
        BH2 = [m2,[-x2,0,0],[0,-yDot2,0]]
    
    
    particles = [BH1,BH2]


    #Soliton parameters are mass, position, velocity and phase (radians)
    soliton1 = [(1-Ratio)*m1,[x1,0,0],[0,yDot1,0], 0]
    soliton2 = [(1-Ratio)*m2,[-x2,0,0],[0,-yDot2,0],0]
    
    solitonC = [10,[0,0,0],[0,0,0],0]

    #solitons = [soliton1,soliton2]

    solitons = [solitonC]
    

else:
    print("Parameter loading skipped in this block.")

## General Initial Settings

In [ ]:
Singe_Mass_Trial = True

if Singe_Mass_Trial:
    
    m = 4
    mS = 0.1
    
    m1 = 20
    m2 = 1

    BH1 = [m1,[-1,0,0],[2,0,0]]
    BH2 = [m2,[-0.5,-2,0],[0,10,0]]


    particles = [BH1,BH2]


    #Soliton parameters are mass, position, velocity and phase (radians)
    
    soliton1 = [10, [0,0,0],[0,0,0], 0]


    solitons = [soliton1]
    #solitons = []


    NumSol = len(solitons)
else:
    print("Parameter loading skipped in this block.")

In [ ]:
NumSol = len(solitons)

print(NumSol)

# Set Axion Mass (SI)

In [ ]:
axion_mass = 1e-22 *1.783e-36 #kg

# Set Simulation Parameters

In [ ]:
# Set number of threads to target
num_threads = multiprocessing.cpu_count()
print("Available CPU threads for this run: ",num_threads)

# Set units for soliton parameters
s_mass_unit = ''     #Accepted units: 'kg', 'solar_masses', 'M_solar_masses', and '' for dimensionless units
s_position_unit = '' #Accepted units: 'm', 'km', 'pc', 'kpc', 'Mpc', 'ly', and '' for dimensionless units
s_velocity_unit = '' #Accepted units: 'm/s', 'km/s', 'km/h', and '' for dimensionless units

# Rules ditto.
m_mass_unit = ''
m_position_unit = ''
m_velocity_unit = ''

# Set box size and resolution
length = 8 # 1 code unit is ~38 kpc x (1e-22/m_a)^0.5
length_units = ''  # Accepted units: 'm', 'km', 'pc', 'kpc', 'Mpc', 'ly', and '' for dimensionless units.
resol= 128 # It is recommended to check the upper bound on soliton mass for a given box size and resolution
duration = 0.4 #1 code unit is ~70 Gyr (independent of axion mass assumption)
duration_units = ''  # Accepted units: 's', 'yr', 'kyr', 'Myr', and '' for dimensionless units
start_time = 0.05 # Should be given in the same units as duration. 

#Data to save
#0
save_rho = False # Saves density data for entire 3D simulation grid
#1
save_psi = False # Saves full complex field data for entire 3D simulation grid
#2
save_plane = True # Saves density data for plane z = 0
#3
save_energies = True # Saves integrated gravitational, kinetic and total energies as lists
#4
save_line = False # Saves density data for line y = 0, z = 0. Useful for examining intereference patterns. 
## FW
#5
save_testmass = True # Saves trajectory and 3-velocity of a test mass thrown into the system. 
                    #Please see PyUL_Floater's code header for documentation.
#Formats to save
hdf5 = False
npz = False
npy = True

step_factor = 1 # Change this to a larger number if velocities are sufficiently low that constraint on timestep can be relaxed. 
save_number = 750    # Choose number of 'frames' to save. Note that, depending on resolution, this could require significant disk space.


save_options = [save_rho,save_psi,save_plane,save_energies,save_line,save_testmass]

# Test Mass Energy Computation Snippets:

In [ ]:
# Removes Solitons in the IV and replaces them with a uniform wavefunction 
# with given probability amplitude (code unit).

Uniform = True
Density = 0.1



# Run:

In [ ]:

try:
    TimeStamp = PyUltraLight_Floater_Advanced.evolveF(num_threads, length, length_units, resol, 
            duration, duration_units, step_factor, save_number, save_options,
           save_path, npz, npy, hdf5, s_mass_unit, s_position_unit, s_velocity_unit, solitons,
           start_time, m_mass_unit, m_position_unit, m_velocity_unit, particles, NS,GridInfinityFix,Method,
                                           Uniform,Density)
    
except KeyboardInterrupt:
    print("\n Run Interrupted! The ability to resume simulations is under development.")

# Data Loading:


In [ ]:
output_animated = 1
# 0 for all contours plotted on a single graph (useful when total number of saves is <=10), 
# 1 for an animation of plane ofa density contours (useful when number of saves is large, may take some time),
# 2 for plot of energies over time, 
# 3 for animation of line along axis of symmetry (useful for studying interference patterns).
save_plots = 0
# 0 to display in this window without saving,
# 1 to save as well (will save in 'Visualisations' directory).

#######################################################################

with open('{}{}'.format(save_path, '/timestamp.txt'), 'r') as timestamp:
    ts = timestamp.read()

loc = save_path + '/' + ts

import time   
import warnings 
warnings.filterwarnings("ignore")
plt.ioff()

EndNum = 0
data = []


TMdata = []

for x in np.arange(0,save_number+1,1):
#for x in np.arange(0,550,1):    
    
    try:
        data.append(np.load('{}{}{}{}'.format(loc, '/plane_#', x, '.npy')))
        TMdata.append(np.load('{}{}{}{}'.format(loc, '/TM_#', x, '.npy')))
        EndNum += 1
    except FileNotFoundError:
        
        TimeStamp = ts
        print("Run incomplete or the storage is corrupt.\n")

        break
        
print("Loaded", EndNum, "Data Entries")

# 2D Trajectory Plot

In [ ]:
try:
    a = TimeStamp
except NameError:
    TimeStamp = 'Debug'

plt.clf()

fig = plt.figure(figsize=(24, 24))
# debug
import numexpr as ne
import matplotlib as mpl

Zoom2D = 1
Boundary = length/(2*Zoom2D)

ax = fig.add_subplot(111)
mpl.style.use('seaborn')

if Parabola:
    yParaBola = np.linspace(-length/(2),length/(2),resol)
    xParaBola = ne.evaluate("yParaBola**2/(4*c0)-c0")

    plt.plot(xParaBola,yParaBola)

    plt.plot(-xParaBola,yParaBola)
    
'''  
if Circular:
    
    tCirc = np.linspace(0,2*np.pi,200)

    plt.plot(x1*np.cos(tCirc),x1*np.sin(tCirc),'-',color = (1,0.5,0,0.1))
    plt.plot(x2*np.cos(tCirc),x2*np.sin(tCirc),'-',color = (1,0,0.5,0.1))
'''


plt.xlim([-Boundary,Boundary])
plt.ylim([-Boundary,Boundary])


ax.set_aspect('equal', adjustable='box')

for i in range(EndNum):
    
    
    TMStateLoc = TMdata[i]
    
    if i == 0:
        for particleID in range(len(particles)):
        
            ColorID = particleID/len(particles) # 0, 0.5
        
            TMx = TMStateLoc[int(6*particleID)]
            TMy = TMStateLoc[int(6*particleID+1)]
            plt.plot([TMx],[TMy],'ko')
        
    
    Alpha = 0.3+7*i/(10*EndNum)
    
    for particleID in range(len(particles)):
        
        ColorID = 0.2 # 0, 0.5
        
        TMx = TMStateLoc[int(6*particleID)]
        TMy = TMStateLoc[int(6*particleID+1)]
        plt.plot([TMx],[TMy],'.',color=(0.5,1*ColorID,0.6-ColorID,Alpha))
        
ax.grid(True)



textstr = '\n'.join((
    TimeStamp,
    r'Resolution: $%.0f^3$' % (resol, ),
    r'Box Length: %.0f' % (length, ),
    r'Method: %.0f @ %.0f Steps' % (Method,NS)))

# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords
ax.text(0, 0, textstr, transform=ax.transAxes, fontsize=18,
        verticalalignment='bottom', bbox=props)

plt.show()

# 2D Animation Panel

In [ ]:
mss = 0
try:
    VTimeStamp = TimeStamp
except NameError:
    VTimeStamp = str('Debug')

AnimName = '{}{}{}'.format("Anim2D_",VTimeStamp,'.mp4')

print("Saving ",AnimName)

plot_axis = 'z' #axis that is set to zero
plt.ioff()
fig0, ax0 = plt.subplots(figsize=(20, 10), dpi=50)
ax0.set_aspect('equal')

data0 = np.log(data)
planemax = np.max(data0)
planemin = np.min(data0)

levels = np.linspace(planemin, planemax, 50)
PlotRange = np.linspace(-length/2, length/2,resol)

FPS = 30


def animate0(i):
    ax0.cla()
    ax0.set_aspect('equal')
    ax0.get_xaxis().set_ticks([])
    ax0.get_yaxis().set_ticks([])
    ax0.contour(PlotRange,PlotRange,data0[i], levels=levels, vmin=planemin, vmax=planemax, cmap='coolwarm')
    #ax0.imshow(data[i])
    
    TMStateLoc = TMdata[i]
    for particleID in range(len(particles)):
        TMx = TMStateLoc[int(6*particleID)]
        TMy = TMStateLoc[int(6*particleID+1)]
        TMz = TMStateLoc[int(6*particleID+2)]
        
        Vx = TMStateLoc[int(6*particleID+3)]
        Vy = TMStateLoc[int(6*particleID+4)]
        Vz = TMStateLoc[int(6*particleID+5)]
        ax0.plot([TMy],[TMx],'ko')
        ax0.quiver([TMy],[TMx],[Vy],[Vx])
        
    fig0.suptitle('{}{}{}'.format('Logarithmic Mass Density - plane ', plot_axis, '=0'), fontsize = 15)
    ax0.text(0.90, 1.1, '{}{}'.format('Snapshot # ', i), horizontalalignment='center', verticalalignment='center', transform=ax0.transAxes)
    
    if i%FPS == 0 and i!= 0:
        print('Animated %.0f seconds out of %.2f seconds of data.' % (i/FPS, EndNum/FPS))
        

interval = 0.15 #in seconds
ani0 = matplotlib.animation.FuncAnimation(fig0,animate0,EndNum,interval=interval*1e+3,blit=False)

Writer = matplotlib.animation.writers['ffmpeg']

writer = Writer(fps=FPS, metadata=dict(artist='PyUltraLightF'))

ani0.save(AnimName, writer=writer)

from IPython.display import HTML
animated_plot0 = HTML(ani0.to_jshtml())


fig0.clear()
display(animated_plot0) 

In [ ]:
save_number

# 3D Animation Panel

In [ ]:
try:
    VTimeStamp = TimeStamp
except NameError:
    VTimeStamp = str('Debug')

AnimName = '{}{}{}'.format("Anim3D_",VTimeStamp,'.mp4')

print("Saving ",AnimName)

# Initialization
fig = plt.figure(figsize=(8, 8))
ax = fig.gca(projection='3d')

ax.view_init(90, 0)

PlotRange = np.linspace(-length/2, length/2,resol)

zoom = 1

FPS = 24 

x3D, y3D = np.meshgrid(PlotRange,PlotRange)

def animate(i):
    fig.clear()
    global FPS
    
    TMStateLoc = TMdata[i]
    ax = fig.gca(projection='3d')

    ax.view_init(90, 0)

    ax.set_zlim3d(-length/2, length/2)
    ax.set_ylim3d(-length/(2*zoom), length/(2*zoom))                    
    ax.set_xlim3d(-length/(2*zoom), length/(2*zoom))

    ax.set_xlabel('x')
    ax.set_ylabel('y')


    if NumSol != 0:
        ax.contour(PlotRange,PlotRange,(data[i]),zdir='z', offset=0, cmap=cm.coolwarm)
        
        #ax.plot_surface(x3D,y3D,data[i],cmap=cm.coolwarm)
    
    for particleID in range(len(particles)):
        TMx = TMStateLoc[int(6*particleID)]
        TMy = TMStateLoc[int(6*particleID+1)]
        TMz = TMStateLoc[int(6*particleID+2)]
        
        Vx = TMStateLoc[int(6*particleID+3)]
        Vy = TMStateLoc[int(6*particleID+4)]
        Vz = TMStateLoc[int(6*particleID+5)]
        ax.plot([TMy],[TMx],[TMz],'ko')
        ax.quiver([TMy],[TMx],[TMz],[Vy],[Vx],[Vz],length=length/(resol*zoom), normalize=False)
    #ax.plot([TMS[i,0]],[TMS[i,1]],[TMS[i,2]],'ro')
    
    if i%FPS == 0 and i!= 0:
        print('Animated %.0f seconds out of %.2f seconds of data.' % (i/FPS, EndNum/FPS))
    plt.draw() 

Writer = matplotlib.animation.writers['ffmpeg']

writer = Writer(fps=FPS, metadata=dict(artist='PyUltraLightF'))

interval = 0.05 #in seconds

ani = matplotlib.animation.FuncAnimation(fig,animate,EndNum,interval=interval*1e+3,blit=False)

ani.save(AnimName, writer=writer)
 

In [ ]:
from IPython.display import HTML

HTML("""
    <video alt="test" controls>
        <source src="Anim3D_Debug.mp4" type="video/mp4">
    </video>
""")


# Test Mass Energy Plot (Extra Options Available for 2-Body Only)

In [ ]:
plt.clf()
# 2Body Energy Savr 
m1 = 20
m2 = 1
KS = np.zeros(int(EndNum))
PS = np.zeros(int(EndNum))

for i in range(int(EndNum)):
    
    Data = TMdata[i]
    
    if len(particles) == 2:
        r = Data[0:2] - Data[6:8]

        rN = np.linalg.norm(r)
    
        PS[i] = -1*m1*m2/rN
    
    for particleID in range(len(particles)):
        Vx = Data[int(6*particleID+3)]
        Vy = Data[int(6*particleID+4)]
        Vz = Data[int(6*particleID+5)]
        
        KS[i] = KS[i] + 1/2*ML[particleID]*(Vx**2+Vy**2+Vz**2) 

Tp = np.linspace(0,EndNum,EndNum)    

plt.figure()
plt.plot(Tp,KS,label = '$\sum E_k$')

if len(particles) == 2:
    plt.plot(Tp,PS,label = '$\sum E_p$')
    plt.plot(Tp,PS+KS,'-', label = '$E_k + E_p$')
#plt.plot(Tp,0*Tp,'k--')
plt.title('Energy of Test Masses')
plt.xlabel('Snapshot Number')
plt.ylabel('$\mathcal{M}\mathcal{L}^2\mathcal{T}^{-2}$')

plt.legend()
plt.show()
    


# Advanced Energy Processing Unit

In [ ]:
fig = plt.figure(figsize=(8, 4.5))

egylist = np.load('{}{}'.format(loc, '/egylist.npy')).tolist()
egpcmlist = np.load('{}{}'.format(loc, '/egpcmlist.npy')).tolist()
egpsilist = np.load('{}{}'.format(loc, '/egpsilist.npy')).tolist()
ekandqlist = np.load('{}{}'.format(loc, '/ekandqlist.npy')).tolist()
mtotlist = np.load('{}{}'.format(loc, '/masseslist.npy')).tolist()


plt.plot(egylist,'g-',label='Total ULDM Energy')
plt.plot(egpcmlist,'c-.',label='$E_{GP}$ (ULD Potential Due to Test Masses)')
plt.plot(egpsilist,'m-.',label='$E_{GP}$ (ULD Potential Due to Self-Interaction)')
plt.plot(ekandqlist,label='ULD $E_{K}+E_{Q}$')


plt.plot(Tp,KS,'y--',label = '$\sum E_k$ for Test Masses')

if len(particles) == 2:
    plt.plot(Tp,PS,'r--',label = '$\sum E_p$ for Test Masses')
    plt.plot(Tp,PS+KS,'b-', label = 'Total Energy of Test Masses')
    
    plt.plot(Tp,PS+KS+egylist,'k-', label = 'Total Energy of Entire System')
#plt.plot(Tp,0*Tp,'k--')

plt.legend(loc='best', bbox_to_anchor=(0.5, 1.3), frameon=False, ncol=2)

plt.xlabel('Snapshot Number')
plt.ylabel('$\mathcal{M}\mathcal{L}^2\mathcal{T}^{-2}$')

plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

plt.savefig('energy_diagram.eps', format='eps', dpi=1000)
plt.savefig('energy_diagram.jpg', format='jpg', dpi=300)


plt.xlim([0,300])
plt.ylim([-400,150])
plt.show()
